# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    unique_words = list(set(text))
      
    vocab_to_int = {}
    int_to_vocab = {}
    
    for index, word in enumerate(unique_words):
        vocab_to_int[word] = index
        int_to_vocab[index] = word

    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {
        "."  : "||Period||",
        ","  : "||Comma||",
        "\"" : "||Quotation_Mark||",
        ";"  : "||Semicolon||",
        "?"  : "||Question_Mark||",
        "!"  : "||Exclamation_Mark||",
        "("  : "||Left_Parentheses||",
        ")"  : "||Right_Parentheses||",
        "--" : "||Dash||",
        "\n" : "||Return||"
    }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.5.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None, None], name="input")
    targets = tf.placeholder(tf.int32, shape=[None, None], name="target")
    learning_rate = tf.placeholder(tf.float32, shape=None, name="learning_rate")
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm_layers = 4
    keep_prob = tf.constant(0.8, name="keep_prob")
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_normal((vocab_size, embed_dim), -1, 1), name="embedding")
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=inputs.dtype)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embeded_data = get_embed(input_data, vocab_size, embed_dim)
    output, final_state = build_rnn(cell, embeded_data)
    
    logits = tf.contrib.layers.fully_connected(output, vocab_size, activation_fn=None)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def printDeets(int_text, batch_size, seq_length):
    print("Len of input: {}".format(len(int_text)))
    print("Batch size: {}   Seq_Len: {}".format(batch_size, seq_length))
    print(int_text)
    
def arrayDeets(arr):
    print("Size: {}  Shape: {}".format(len(arr), arr.shape))
    print(arr)

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """    
    # TODO: Implement Function
    words_per_batch = batch_size * seq_length
    n_batches = len(int_text) // words_per_batch
    
    # Keep only enough words to make full batches
    truncated_int_text = np.asarray(int_text[:n_batches * words_per_batch])

    # Getting the total inputs and targets array
    inputs = np.array([int_text[:words_per_batch * n_batches]])
    # Rotates everything to the left by 1. This means for the nth element in the input array will be the nth - 1
    # element in targets array
    targets = np.roll(inputs, -1)

    # The arrays have someone become nested. Doing this to remove the nesting of the arrays
    inputs = inputs[0]
    targets = targets[0]
    
    # Splitting the array into batches
    inputs_batches = np.reshape(inputs, (batch_size, -1))
    targets_batches = np.reshape(targets, (batch_size, -1))

    # Forming the sub arrays required for the sliding windows for batches
    num_of_sliding_windows = inputs_batches.shape[1] / seq_length
    windowed_inputs = np.asarray(np.split(inputs_batches, num_of_sliding_windows, axis = 1), np.int32)
    windowed_targets = np.asarray(np.split(targets_batches, num_of_sliding_windows, axis = 1), np.int32)
    
    # Joining up the inputs and targets sub batches into a batch
    batches = np.asarray(list(zip(windowed_inputs, windowed_targets)))

    return batches

''' IGNORE THIS PLEASE. KEEPING AS BACK UP 
       
    # Creating an empty array of the shape required with the element type being an int 
    batches = np.empty((n_batches, 2, batch_size, seq_length), np.int32)
 
    # Working the way through the batches
    for b in range(0, len(batches)):
        # Getting the current working batch
        _batch = batches[b]
        
        # Getting the arrays containing the inputs and the targets from the current batch
        _inputs = _batch[0]
        _targets = _batch[1]
        
        for b_s in range(0, batch_size):
            for s_l in range(0, seq_length):
                idx = ((b * seq_length) + s_l + (b_s * (words_per_batch - 1)))
                _inputs[b_s][s_l] = int_text[ idx ]
        
        for b_s in range(0, batch_size):
            for s_l in range(0, seq_length):
                # Mod is used to go back to the start when the end of the array is reached
                idx = ((b * seq_length) + s_l + (b_s * words_per_batch) + 1) % (n_batches * words_per_batch) 
                _targets[b_s][s_l] = int_text[ idx ]
        
        # Updating the current batch with the inputs and the targets
        _batch[0] = _inputs
        _batch[1] = _targets
        
        # Updating the batches array with the data from the current batch
        batches[b] = _batch
       
    return batches
    
print(get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2))
'''

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 2200
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 2048
# Embedding Dimension Size
embed_dim = 2048
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.0005
# Show stats for every n number of batches
show_every_n_batches = 30

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/13   train_loss = 8.822
Epoch   2 Batch    4/13   train_loss = 5.926
Epoch   4 Batch    8/13   train_loss = 5.964
Epoch   6 Batch   12/13   train_loss = 5.870
Epoch   9 Batch    3/13   train_loss = 5.594
Epoch  11 Batch    7/13   train_loss = 5.382
Epoch  13 Batch   11/13   train_loss = 5.186
Epoch  16 Batch    2/13   train_loss = 5.004
Epoch  18 Batch    6/13   train_loss = 4.907
Epoch  20 Batch   10/13   train_loss = 4.689
Epoch  23 Batch    1/13   train_loss = 4.739
Epoch  25 Batch    5/13   train_loss = 4.514
Epoch  27 Batch    9/13   train_loss = 4.396
Epoch  30 Batch    0/13   train_loss = 4.405
Epoch  32 Batch    4/13   train_loss = 4.214
Epoch  34 Batch    8/13   train_loss = 4.236
Epoch  36 Batch   12/13   train_loss = 4.243
Epoch  39 Batch    3/13   train_loss = 4.344
Epoch  41 Batch    7/13   train_loss = 4.024
Epoch  43 Batch   11/13   train_loss = 3.950
Epoch  46 Batch    2/13   train_loss = 3.996
Epoch  48 Batch    6/13   train_loss = 4.033
Epoch  50 

Epoch 422 Batch    4/13   train_loss = 0.445
Epoch 424 Batch    8/13   train_loss = 0.461
Epoch 426 Batch   12/13   train_loss = 0.423
Epoch 429 Batch    3/13   train_loss = 0.390
Epoch 431 Batch    7/13   train_loss = 0.384
Epoch 433 Batch   11/13   train_loss = 0.370
Epoch 436 Batch    2/13   train_loss = 0.355
Epoch 438 Batch    6/13   train_loss = 0.370
Epoch 440 Batch   10/13   train_loss = 0.355
Epoch 443 Batch    1/13   train_loss = 0.354
Epoch 445 Batch    5/13   train_loss = 0.337
Epoch 447 Batch    9/13   train_loss = 0.343
Epoch 450 Batch    0/13   train_loss = 0.335
Epoch 452 Batch    4/13   train_loss = 0.365
Epoch 454 Batch    8/13   train_loss = 0.354
Epoch 456 Batch   12/13   train_loss = 0.346
Epoch 459 Batch    3/13   train_loss = 0.330
Epoch 461 Batch    7/13   train_loss = 0.333
Epoch 463 Batch   11/13   train_loss = 0.340
Epoch 466 Batch    2/13   train_loss = 0.332
Epoch 468 Batch    6/13   train_loss = 0.342
Epoch 470 Batch   10/13   train_loss = 0.341
Epoch 473 

Epoch 844 Batch    8/13   train_loss = 0.343
Epoch 846 Batch   12/13   train_loss = 0.335
Epoch 849 Batch    3/13   train_loss = 0.319
Epoch 851 Batch    7/13   train_loss = 0.323
Epoch 853 Batch   11/13   train_loss = 0.330
Epoch 856 Batch    2/13   train_loss = 0.322
Epoch 858 Batch    6/13   train_loss = 0.333
Epoch 860 Batch   10/13   train_loss = 0.332
Epoch 863 Batch    1/13   train_loss = 0.340
Epoch 865 Batch    5/13   train_loss = 0.322
Epoch 867 Batch    9/13   train_loss = 0.329
Epoch 870 Batch    0/13   train_loss = 0.323
Epoch 872 Batch    4/13   train_loss = 0.354
Epoch 874 Batch    8/13   train_loss = 0.343
Epoch 876 Batch   12/13   train_loss = 0.334
Epoch 879 Batch    3/13   train_loss = 0.319
Epoch 881 Batch    7/13   train_loss = 0.323
Epoch 883 Batch   11/13   train_loss = 0.329
Epoch 886 Batch    2/13   train_loss = 0.322
Epoch 888 Batch    6/13   train_loss = 0.332
Epoch 890 Batch   10/13   train_loss = 0.332
Epoch 893 Batch    1/13   train_loss = 0.340
Epoch 895 

Epoch 1260 Batch    0/13   train_loss = 0.321
Epoch 1262 Batch    4/13   train_loss = 0.353
Epoch 1264 Batch    8/13   train_loss = 0.341
Epoch 1266 Batch   12/13   train_loss = 0.332
Epoch 1269 Batch    3/13   train_loss = 0.317
Epoch 1271 Batch    7/13   train_loss = 0.321
Epoch 1273 Batch   11/13   train_loss = 0.328
Epoch 1276 Batch    2/13   train_loss = 0.320
Epoch 1278 Batch    6/13   train_loss = 0.331
Epoch 1280 Batch   10/13   train_loss = 0.330
Epoch 1283 Batch    1/13   train_loss = 0.338
Epoch 1285 Batch    5/13   train_loss = 0.320
Epoch 1287 Batch    9/13   train_loss = 0.327
Epoch 1290 Batch    0/13   train_loss = 0.320
Epoch 1292 Batch    4/13   train_loss = 0.352
Epoch 1294 Batch    8/13   train_loss = 0.341
Epoch 1296 Batch   12/13   train_loss = 0.332
Epoch 1299 Batch    3/13   train_loss = 0.317
Epoch 1301 Batch    7/13   train_loss = 0.321
Epoch 1303 Batch   11/13   train_loss = 0.327
Epoch 1306 Batch    2/13   train_loss = 0.320
Epoch 1308 Batch    6/13   train_l

Epoch 1673 Batch    1/13   train_loss = 0.337
Epoch 1675 Batch    5/13   train_loss = 0.319
Epoch 1677 Batch    9/13   train_loss = 0.327
Epoch 1680 Batch    0/13   train_loss = 0.320
Epoch 1682 Batch    4/13   train_loss = 0.351
Epoch 1684 Batch    8/13   train_loss = 0.340
Epoch 1686 Batch   12/13   train_loss = 0.331
Epoch 1689 Batch    3/13   train_loss = 0.316
Epoch 1691 Batch    7/13   train_loss = 0.320
Epoch 1693 Batch   11/13   train_loss = 0.327
Epoch 1696 Batch    2/13   train_loss = 0.319
Epoch 1698 Batch    6/13   train_loss = 0.330
Epoch 1700 Batch   10/13   train_loss = 0.329
Epoch 1703 Batch    1/13   train_loss = 0.337
Epoch 1705 Batch    5/13   train_loss = 0.319
Epoch 1707 Batch    9/13   train_loss = 0.327
Epoch 1710 Batch    0/13   train_loss = 0.320
Epoch 1712 Batch    4/13   train_loss = 0.351
Epoch 1714 Batch    8/13   train_loss = 0.340
Epoch 1716 Batch   12/13   train_loss = 0.331
Epoch 1719 Batch    3/13   train_loss = 0.316
Epoch 1721 Batch    7/13   train_l

Epoch 2086 Batch    2/13   train_loss = 0.318
Epoch 2088 Batch    6/13   train_loss = 0.329
Epoch 2090 Batch   10/13   train_loss = 0.328
Epoch 2093 Batch    1/13   train_loss = 0.336
Epoch 2095 Batch    5/13   train_loss = 0.319
Epoch 2097 Batch    9/13   train_loss = 0.326
Epoch 2100 Batch    0/13   train_loss = 0.319
Epoch 2102 Batch    4/13   train_loss = 0.351
Epoch 2104 Batch    8/13   train_loss = 0.339
Epoch 2106 Batch   12/13   train_loss = 0.331
Epoch 2109 Batch    3/13   train_loss = 0.316
Epoch 2111 Batch    7/13   train_loss = 0.319
Epoch 2113 Batch   11/13   train_loss = 0.326
Epoch 2116 Batch    2/13   train_loss = 0.318
Epoch 2118 Batch    6/13   train_loss = 0.329
Epoch 2120 Batch   10/13   train_loss = 0.328
Epoch 2123 Batch    1/13   train_loss = 0.336
Epoch 2125 Batch    5/13   train_loss = 0.319
Epoch 2127 Batch    9/13   train_loss = 0.326
Epoch 2130 Batch    0/13   train_loss = 0.319
Epoch 2132 Batch    4/13   train_loss = 0.351
Epoch 2134 Batch    8/13   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [31]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return (
        loaded_graph.get_tensor_by_name(name="input:0"), loaded_graph.get_tensor_by_name("initial_state:0"),
        loaded_graph.get_tensor_by_name( "final_state:0"), loaded_graph.get_tensor_by_name("probs:0")
    )


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [70]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = probabilities.argmax(axis = 0)
    word = int_to_vocab[index] 
    
    return word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [73]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson:(drunk, emotional) ah, i don't care to think you are.
homer_simpson:(loud) i'll have ya if we got a date with a friend!
moe_szyslak: yeah, that's the little more sensitivity, homer.
homer_simpson:(bitter) na ya off a bunch) /(lowers glass).
carl_carlson: i hear my-- my" flaming, a little japanese /.
lisa_simpson:(sotto to homer) that's the people!
moe_szyslak: uh, yeah. here ya ever trusted.
moe_szyslak:(looking) you owe my" going."
bart_simpson: yeah, this is startin' to hurt me to a woman.
moe_szyslak: love off, dad.
homer_simpson:(loud) you know, i had the keys.
moe_szyslak:(struggling noises) oh, sweet mother, bart.
homer_simpson:(getting business) well, i knew.
homer_simpson:(to moe) see about the beer?
marge_simpson: the trolls! oh, i don't know. what do you gonna do everything.
moe_szyslak:(


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.